In [1]:
import geopandas as gpd
import tempun
import matplotlib.pyplot as plt
import pickle

In [2]:
LIST = gpd.read_parquet("../data/large_data/LIST_v0-4.parquet")

In [3]:
(LIST["not_before"] <= LIST["not_after"]).sum()

206421

In [4]:
LIRE = LIST[
    (LIST["geotemporal?"])
    & (LIST["withinRE?"])
    & ((LIST["not_after"]<=350)
    | (LIST["not_before"]>=-50))]

In [5]:
len(LIRE)

203466

In [9]:
print(((LIRE["EDCS-ID"].notnull()) & (LIRE["EDH-ID"].notnull())).sum())

62365


In [6]:
print(LIRE["EDH-ID"].isnull().sum())
print(LIRE["EDH-ID"].notnull().sum())


138597
64869


In [8]:
print(((LIRE["EDCS-ID"].isnull()) & (LIRE["EDH-ID"].notnull())).sum())


2504


In [9]:
LIRE.drop(columns=["withinRE?"], inplace=True)

/var/folders/zg/zvg9y3rs7j527jxfq9sc2xqc0000gn/T/ipykernel_30377/2295483491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LIRE.drop(columns=["withinRE?"], inplace=True)


In [11]:
LIRE.to_parquet("../data/large_data/LIRE_v2-1.parquet")

In [10]:
LIRE.to_file("../data/large_data/LIRE_v2-1.geojson", driver="GeoJSON")